# <center>NASDAQ Trades DATA DESCRIPTION</center>

In [55]:
df = pd.read_csv('{0}daily.csv'.format(COMPANY))
df

,year,month,day,timestamp,seqnum,mktcenter,price,shares,salescondition,canceled,dottchar,issuechar,msgseqnum,originalmsgseqnum,submkt
0,2008,10,27,2008-10-27 14:00:38,1121037,U,363.0000,50,@,False,F,?,4644,0,NaN
1,2008,10,28,2008-10-28 15:58:47,511170121,u,903.0000,40,@,False,F,S,27654,0,NaN
2,2008,10,29,2008-10-29 11:51:50,408314384,u,918.4300,10000,Z,False,F,S,22736,0,NaN
3,2008,10,31,2008-10-31 10:03:55,894147032,u,1000.0000,86,@,False,F,S,51258,0,NaN
4,2008,11,3,2008-11-03 09:32:21,388462522,u,864.0000,5,@,False,F,S,21725,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,2021,7,23,2021-07-23 15:36:55.416,41886656,u,292.0000,1,@,False,F,S,463541,0,NaN
2560,2021,7,26,2021-07-26 15:56:38.489,77399777,u,220.5000,15,@,False,F,S,1476697,0,NaN
2561,2021,7,27,2021-07-27 15:32:52.803,60054657,u,216.0000,1,@,False,F,S,710210,0,NaN
2562,2021,7,28,2021-07-28 15:54:58.119,21450823,u,214.0137,4,@,False,F,S,261639,0,NaN


### Data Description:
- <b>year</b>: given year, from 2008 to 2021 (14 years total)
- <b>month</b>: given month, 12 possible values
- <b>day</b>: given day, 31 possible values
- <b>timestamp</b>: original concatenation of year, month, day, and time of transaction. Should be dropped - no longer necessary.     
- <b>seqnum</b>: NASDAQ sequence number, different for every record. Should be dropped. 
- <b>mktcenter</b>: market center on which trade was executed. Every value should be made uppercase. Possible values: 
    - 'U' = 
- <b>price</b>: price at which the trade was executed. 
- <b>shares</b>: quantity, number of shares for which the trade was executed. 
- <b>salescondition</b>: sales condition. All white spaces should be deleted. Possible values: 
    - '@   ' = Regular settlement
    - 'C' = Cash settlement
    - 'N' = Next day settlement
    - 'R' = Seller settlement
    - 'Z   ' = Sold – Out of Sequence
    - 'T   ' = Extended Hours Trade (documentation page 8)
    - 'U   ' = Extended Hours Trade – Reported Late or Out of Sequence (documentation page 8)
    - 'W   ' = Weighted Average Price
    - 'N T ' = is an error or concatenation of both selling conditions? There is 12 cases at all of 2 symbols. 
    - 'R Z ',
    - 'R T ', 
    - '@  W', 
    - '@  P', 
    - '@ T ', 
    - '@ Z '
    - '<space\>': not applicable (delete)
- <b>canceled</b>: indicates if trade was canceled. Is boolean - True or False. Almost always is False.
- <b>dottchar</b>: DOTT character. Can be either 'F' or 'T'. Does it mean True and False?
    - 'F' = 
    - 'T' = 
- <b>issuechar</b>: can be either '?', 'S', or 'C'.
    - 'S' = System Event Message
    - 'C' = 
    - '?' = 
- <b>msgseqnum</b>: NASDAQ message sequence number, different for most of the records (2537 out of 2564). Should be dropped.   
- <b>originalmsgseqnum</b>: NASDAQ original message sequence number. Should be dropped.
- <b>submkt</b>: sub market center. Usually is empty, null. 

##### Sources: 
 - NASDAQ data on demand official documentation (https://dataondemand.nasdaq.com/docs/index.html#trades)
 - NASDAQ documentation from internet (http://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/nqlastsalespec.pdf)

# <center>CODE</center>

In [51]:
%%time
for year in YEARS: 
    df = pd.read_csv('{0}/{0}{1}.csv'.format(COMPANY, year), 
                     index_col='timestamp', 
                     usecols=['timestamp', 'price', 'shares', 'salescondition', 'mktcenter', 'submkt', 'seqnum', 'canceled', 'dottchar', 'issuechar', 'msgseqnum', 'originalmsgseqnum'], 
                     dtype={"seqnum": "int64",
                            "price": "float64", 
                            "shares": "int64", 
                            "salescondition": "object",
                            "canceled": "bool",
                            "dottchar": "object",
                            "issuechar": "object",
                            "msgseqnum": "int",
                            "originalmsgseqnum": "int",
                            "mktcenter": "object", 
                            "submkt": "object"})
    df.index = df.index.sort_values()
    df.index = pd.to_datetime(df.index)
    df = df.groupby([df.index.year.values, df.index.month.values, df.index.day.values]).apply(pd.Series.tail,1)
    df.to_csv('{0}/{0}{1}aggregated_daily.csv'.format(COMPANY, year))

Wall time: 2.25 s


In [52]:
HEADER = ['year', 
          'month', 
          'day', 
          'timestamp', 
          'seqnum', 
          'mktcenter', 
          'price', 
          'shares', 
          'salescondition', 
          'canceled', 
          'dottchar', 
          'issuechar', 
          'msgseqnum', 
          'originalmsgseqnum', 
          'submkt']

In [53]:
first = True
for year in YEARS:
    df = pd.read_csv('{0}/{0}{1}aggregated_daily.csv'.format(COMPANY, year))
    if first: 
        df.to_csv('{0}daily.csv'.format(COMPANY), header=HEADER, index = False)
        first = False
    else:
        df.to_csv('{0}daily.csv'.format(COMPANY), mode='a', header=False, index = False)

In [55]:
df = pd.read_csv('{0}daily.csv'.format(COMPANY))
df

,year,month,day,timestamp,seqnum,mktcenter,price,shares,salescondition,canceled,dottchar,issuechar,msgseqnum,originalmsgseqnum,submkt
0,2008,10,27,2008-10-27 14:00:38,1121037,U,363.0000,50,@,False,F,?,4644,0,NaN
1,2008,10,28,2008-10-28 15:58:47,511170121,u,903.0000,40,@,False,F,S,27654,0,NaN
2,2008,10,29,2008-10-29 11:51:50,408314384,u,918.4300,10000,Z,False,F,S,22736,0,NaN
3,2008,10,31,2008-10-31 10:03:55,894147032,u,1000.0000,86,@,False,F,S,51258,0,NaN
4,2008,11,3,2008-11-03 09:32:21,388462522,u,864.0000,5,@,False,F,S,21725,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,2021,7,23,2021-07-23 15:36:55.416,41886656,u,292.0000,1,@,False,F,S,463541,0,NaN
2560,2021,7,26,2021-07-26 15:56:38.489,77399777,u,220.5000,15,@,False,F,S,1476697,0,NaN
2561,2021,7,27,2021-07-27 15:32:52.803,60054657,u,216.0000,1,@,False,F,S,710210,0,NaN
2562,2021,7,28,2021-07-28 15:54:58.119,21450823,u,214.0137,4,@,False,F,S,261639,0,NaN


In [59]:
df.describe()

,year,month,day,seqnum,price,shares,msgseqnum,originalmsgseqnum,submkt
count,2564.000000,2564.000000,2564.000000,2.564000e+03,2564.000000,2564.000000,2.564000e+03,2564.000000,0.0
mean,2015.609204,6.490640,15.733229,1.631935e+07,182.883106,128.548362,8.021212e+04,219.276911,NaN
std,3.417521,3.452751,8.795539,4.686896e+07,74.000728,951.210879,1.155985e+05,3583.495076,NaN
min,2008.000000,1.000000,1.000000,7.144000e+03,1.650000,1.000000,4.000000e+00,0.000000,NaN
25%,2013.000000,3.000000,8.000000,2.580193e+06,148.188750,7.000000,1.501375e+04,0.000000,NaN
50%,2016.000000,6.000000,16.000000,1.111765e+07,166.440000,25.000000,4.801400e+04,0.000000,NaN
75%,2019.000000,10.000000,23.000000,2.048704e+07,199.990000,100.000000,9.482050e+04,0.000000,NaN
max,2021.000000,12.000000,31.000000,1.028051e+09,1080.000000,32500.000000,1.476697e+06,99492.000000,NaN


In [60]:
df.shape

(2564, 15)

In [62]:
df.nunique()

year                   14
month                  12
day                    31
timestamp            2564
seqnum               2564
mktcenter               2
price                1766
shares                245
salescondition         13
canceled                2
dottchar                2
issuechar               2
msgseqnum            2537
originalmsgseqnum      17
submkt                  0
dtype: int64

In [64]:
df['mktcenter'].unique()

array(['U', 'u'], dtype=object)

In [78]:
df['salescondition'].unique()

array(['@   ', 'Z   ', 'T   ', 'U   ', 'R   ', 'W   ', 'N T ', 'R Z ',
       'R T ', '@  W', '@  P', '@ T ', '@ Z '], dtype=object)

In [79]:
df['salescondition'].value_counts()

@       2512
T         22
Z          7
R          4
U          4
@  P       3
W          3
R T        2
@ T        2
R Z        2
@ Z        1
N T        1
@  W       1
Name: salescondition, dtype: int64

In [66]:
df['canceled'].unique()

array([False,  True])

In [67]:
df['dottchar'].unique()

array(['F', 'T'], dtype=object)

In [68]:
df['issuechar'].unique()

array(['?', 'S'], dtype=object)

In [69]:
df['originalmsgseqnum'].unique()

array([    0, 22977, 22903, 58149, 61286, 58954,  6220,  6675,   242,
       53642,  5307,    70, 56820, 99492, 23492, 20584, 65413],
      dtype=int64)

In [70]:
df['submkt'].unique()

array([nan])

In [72]:
df.isnull().sum()

year                    0
month                   0
day                     0
timestamp               0
seqnum                  0
mktcenter               0
price                   0
shares                  0
salescondition          0
canceled                0
dottchar                0
issuechar               0
msgseqnum               0
originalmsgseqnum       0
submkt               2564
dtype: int64

## Final data after dropping unnecessary columns

In [80]:
df = df.drop(['timestamp', 'seqnum', 'msgseqnum', 'originalmsgseqnum', 'submkt'], axis = 1)
df

,year,month,day,mktcenter,price,shares,salescondition,canceled,dottchar,issuechar
0,2008,10,27,U,363.0000,50,@,False,F,?
1,2008,10,28,u,903.0000,40,@,False,F,S
2,2008,10,29,u,918.4300,10000,Z,False,F,S
3,2008,10,31,u,1000.0000,86,@,False,F,S
4,2008,11,3,u,864.0000,5,@,False,F,S
...,...,...,...,...,...,...,...,...,...,...
2559,2021,7,23,u,292.0000,1,@,False,F,S
2560,2021,7,26,u,220.5000,15,@,False,F,S
2561,2021,7,27,u,216.0000,1,@,False,F,S
2562,2021,7,28,u,214.0137,4,@,False,F,S
